In [4]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col,sum,when


spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

In [3]:
Stocks = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "Stocks") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

Stocks.show()

+------------+---------+-------------+-----+
|  stock_name|operation|operation_day|price|
+------------+---------+-------------+-----+
|    Leetcode|     Buy |            1| 1000|
|Corona Masks|     Buy |            2|   10|
|    Leetcode|     Sell|            5| 9000|
|    Handbags|     Buy |           17|30000|
|Corona Masks|     Sell|            3| 1010|
|Corona Masks|     Buy |            4| 1000|
|Corona Masks|     Sell|            5|  500|
|Corona Masks|     Buy |            6| 1000|
|    Handbags|     Sell|           29| 7000|
|Corona Masks|     Sell|           10|10000|
+------------+---------+-------------+-----+



In [8]:
Stocks\
    .withColumn("buy_price",when(col("operation")=='Buy',col("price")))\
    .withColumn("sell_price",when(col("operation")=='Sell',col("price")))\
    .groupBy(col("stock_name"))\
    .agg(sum("buy_price").alias("buy_price"),
         sum("sell_price").alias("sell_price"))\
    .withColumn("capital_gain_loss",col("sell_price")-col("buy_price"))\
    .drop("buy_price","sell_price")\
    .show()

+------------+-----------------+
|  stock_name|capital_gain_loss|
+------------+-----------------+
|    Handbags|           -23000|
|Corona Masks|             9500|
|    Leetcode|             8000|
+------------+-----------------+

